In [35]:
import os
import tensorflow as tf
from tensorflow import keras
import cv2
import mediapipe as mp
import numpy as np


In [36]:
model = tf.keras.models.load_model('/Users/anhhung/Desktop/TaiLieu/DPL/r_p_s/RPS/model1.h5')

# Check its architecture
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_14 (Dropout)        (None, 42)                0         
                                                                 
 dense_21 (Dense)            (None, 20)                860       
                                                                 
 dropout_15 (Dropout)        (None, 20)                0         
                                                                 
 dense_22 (Dense)            (None, 10)                210       
                                                                 
 dense_23 (Dense)            (None, 3)                 33        
                                                                 
Total params: 1,103
Trainable params: 1,103
Non-trainable params: 0
_________________________________________________________________


In [37]:
import cv2
import csv
import mediapipe as mp

In [40]:
# Load the trained model and label mapping
label_name_mapping = {'1': 'paper', '2': 'rock', '3': 'scissors'}

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

# Get the width and height of the video capture
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize mediapipe hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            continue

        # Flip the image horizontally for a later selfie-view display, and convert the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)

        # Draw the rectangle on the frame
        rectangle_width = 500
        rectangle_height = 500
        cv2.rectangle(image, (width - rectangle_width, 0), (width, rectangle_height), (255, 255, 255), 2)

        # To improve performance, optionally mark the image as not writeable to pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                 # Check if hand is within the specified rectangle
                hand_rect_w = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * width
                hand_rect_h = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * height  # Adjusted line

                if hand_rect_w > width - rectangle_width and hand_rect_h < rectangle_height:  # Adjusted condition
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Extract hand landmarks coordinates
                    count = 1
                    coordinates = []
                    for landmark in hand_landmarks.landmark:
                        if count == 1:
                            x0 = landmark.x
                            y0 = landmark.y
                            coordinates.extend([0, 0])
                            count+=1
                        else:
                            x = landmark.x
                            y = landmark.y
                            x = round(x - x0, 5)
                            y = round(y - y0, 5)
                            coordinates.extend([x, y])


                    # Perform the prediction
                    input_data = np.array([coordinates])
                    predicted_probs = model.predict(input_data)[0]
                    predicted_label = np.argmax(predicted_probs) + 1  # Adjust label index to match mapping

                    # Get the corresponding label name from the mapping
                    label_name = label_name_mapping[str(predicted_label)]

                    # Get the predicted probability for the predicted label
                    predicted_prob = predicted_probs[predicted_label - 1]

                    # Check if predicted probability is under 0.75
                    if predicted_prob < 0.75:
                        label_name = 'nothing'

                    # Display the predicted label and probability on the image
                    label_text = f"{label_name}: {predicted_prob:.2f}"
                    cv2.putText(image, label_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


        cv2.imshow("New Window", image)
        
        k = cv2.waitKey(10)

        if k == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

2023-06-08 21:28:56.183 python[22988:299166] _TIPropertyValueIsValid called with 4 on nil context!
2023-06-08 21:28:56.183 python[22988:299166] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-06-08 21:28:56.183 python[22988:299166] Text input context does not respond to _valueForTIProperty:
2023-06-08 21:28:56.183 python[22988:299166] _TIPropertyValueIsValid called with 4 on nil context!
2023-06-08 21:28:56.183 python[22988:299166] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-06-08 21:28:56.183 python[22988:299166] Text input context does not respond to _valueForTIProperty:
2023-06-08 21:28:56.183 python[22988:299166] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_presentFunctionRowItemTextInputViewWithEndpoint:reply:) block performed very slowly (140.72 secs).


1/1 [==============================] - 0s 24ms/step


2023-06-08 21:30:33.345 python[22988:299166] _TIPropertyValueIsValid called with 4 on nil context!
2023-06-08 21:30:33.345 python[22988:299166] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-06-08 21:30:33.345 python[22988:299166] Text input context does not respond to _valueForTIProperty:


1/1 [==============================] - 0s 23ms/step
